In [1]:
import numpy as np
import pandas as pd
from sklearn.decomposition import PCA
from sklearn.model_selection import train_test_split

## 資料前處理-全轉成數值

In [2]:
df = pd.read_csv('repurchase_info.csv')

#做出ICD_10
ICD_10 = []
for i in range(df.shape[0]):
    ICD_10.append(ord(df['illness_code'][i][0]) - 65) #ord('A') = 65
df['ICD_10'] = ICD_10

#cleaning
#client income有很多0, 不知道是否有問題
df = df.drop(['REPURCHASE','INSURED_RK','Policy_RK', 'ICD10_Code_Type', 'CUST_RK','BundleSubtype2', 'illness_code', 'illness_desc', 'DiagnosisCode_DESC', 'claim_settle_dt','POLICY_HOLDER_RK'], axis = 1)

#wealth_level
dic = {'W1':0, 'W2':1, 'W3':2,'W4':3,'W5':4,'W6':5,'W7':6}
for i in range(df.shape[0]):
    df['WEALTH_LEVEL'][i] = dic[df['WEALTH_LEVEL'][i]]
    
#stick_level2
dic = {'S01':0, 'S02':1,'S03':2,'S04':3,'S05':4,'S06':5,'S07':6,'S08':7,'S09':8,'S10':9}
for i in range(df.shape[0]):
    df['stick_level2'][i] = dic[df['stick_level2'][i]]

#cust_group2
dic = {'G0':0, 'G1':1,'G2':2,'G3':3,'G4':4}
for i in range(df.shape[0]):
    df['cust_group2'][i] = dic[df['cust_group2'][i]]
    
df.to_csv('repurchase_numeric.csv', index = False)

/home/chiluen/.local/lib/python3.6/site-packages/ipykernel_launcher.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  app.launch_new_instance()
/home/chiluen/.local/lib/python3.6/site-packages/ipykernel_launcher.py:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/home/chiluen/.local/lib/python3.6/site-packages/ipykernel_launcher.py:26: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


## 資料前處理-轉成dummies

In [104]:
df = pd.read_csv('repurchase_info.csv')

ICD_10 = []
for i in range(df.shape[0]):
    ICD_10.append(df['illness_code'][i][0]) 
df['ICD_10'] = ICD_10

#cleaning
#client income有很多0, 不知道是否有問題
df = df.drop(['REPURCHASE','INSURED_RK','Policy_RK', 'ICD10_Code_Type', 'CUST_RK','BundleSubtype2', 'illness_code', 'illness_desc', 'DiagnosisCode_DESC', 'claim_settle_dt','POLICY_HOLDER_RK'], axis = 1)

#做排序，依照：數值，binary class， multiple class，並且把multiple class做成dummy variable
df = df[['REIMBURSED_YR_TW', 'ternure_m', 'recency_m','CLIENT_INCOME', 'AGE', 'SIN', 'SIN_his', 'REG',
       'REG_his', 'ILP', 'ILP_his', 'AHa', 'AHa_his', 'AHb', 'AHb_his', 'AHc',
       'AHc_his', 'AHd', 'AHd_his', 'VIP','DIGI_FLG', 'TOPCARD', 'GENDER', 'WEALTH_LEVEL','stick_level2','cust_group2','ICD_10']]

dum_list = ['WEALTH_LEVEL','stick_level2','cust_group2','ICD_10']
for i in dum_list:
    dummies = pd.get_dummies(df[i])
    df = pd.concat([df, dummies], axis=1)
    df = df.drop([i], axis = 1)
    
df.to_csv('repurchase_dummies.csv', index = False)

## Training code - dummies edition

In [2]:
## df.shape = (95408, 73)
temp = pd.read_csv('repurchase_info.csv')
df = pd.read_csv('repurchase_dummies.csv')
ans = np.array((temp['REPURCHASE']))

In [3]:
df

,REIMBURSED_YR_TW,ternure_m,recency_m,CLIENT_INCOME,AGE,SIN,SIN_his,REG,REG_his,ILP,...,O,Q,R,S,T,V,W,X,Y,Z
0,28046.55,200,18,475000,76,1,1,1,1,0,...,0,0,0,0,0,0,0,0,0,0
1,871.50,70,70,2850000,71,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
2,2593.50,282,269,0,69,0,0,0,1,0,...,0,0,0,0,0,1,0,0,0,0
3,16537.50,490,237,0,73,0,0,1,1,0,...,0,0,0,0,0,0,0,0,1,0
4,71400.00,191,119,475000,67,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95403,472500.00,343,120,0,57,0,0,1,1,0,...,0,0,0,0,0,0,0,0,0,0
95404,4389.00,321,201,0,53,0,0,1,1,0,...,0,0,0,0,0,1,0,0,0,0
95405,31133.55,263,240,0,54,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
95406,17002.65,317,282,1425000,56,0,0,1,1,0,...,0,0,0,0,0,0,0,0,0,0


In [4]:
##normalization
numeric_list = ['REIMBURSED_YR_TW', 'ternure_m', 'recency_m', 'CLIENT_INCOME', 'AGE']
for i in numeric_list:
    df[i] = (df[i] - df[i].mean())/df[i].std()

#PCA
#pca = PCA(n_components=30)
#df = pca.fit_transform(df)

In [5]:
import torch
from torch.utils.data.dataset import Dataset
from torch.utils.data import DataLoader
import torch.nn as nn


In [6]:
class build_dataset(Dataset):
    def __init__(self, x, y):
        self.x = x
        self.y = torch.LongTensor(y)
        
    def __getitem__(self, index):
        return self.x[index], self.y[index]
    
    def __len__(self):
        return len(self.x)

X_train, X_test, y_train, y_test = train_test_split(np.array(df), ans, test_size=0.2, random_state=0)
repurchase_dataset = build_dataset(X_train, y_train)
repurchase_dataloader = DataLoader(
    dataset = repurchase_dataset,
    batch_size = 100,
    shuffle = True,
    num_workers = 2
)
valid_dataset = build_dataset(X_test, y_test)
valid_dataloader = DataLoader(
    dataset = valid_dataset,
    batch_size = 100,
    shuffle = False,
    num_workers = 2
)

class model(nn.Module):
    def __init__(self, dimension):
        super().__init__()
        
        self.fc = nn.Sequential(
            nn.Linear(dimension, 20),
            nn.BatchNorm1d(20),
            nn.ReLU(),
            
            nn.Linear(20,10),
            nn.BatchNorm1d(10),
            nn.ReLU(),
            
            nn.Linear(10,2),
            nn.Softmax()
        )
        
    def forward(self, x):
        output = self.fc(x)
        return output

In [7]:
## base line: 0.894

total_train_loss = []
total_valid_loss = []

# model setting
dimension = X_train.shape[1]
classifier = model(dimension).cuda()

# parameters
lr = 0.001
num_epochs = 30
CrossEntropy = nn.CrossEntropyLoss()
optim = torch.optim.Adam(classifier.parameters(), lr = lr)


for epoch in range(num_epochs):
    
    train_acc = 0
    train_loss = 0
    valid_acc = 0
    valid_loss = 0
    train_counter = 0
    valid_counter = 0
    
    classifier.train()
    for i, data in enumerate(repurchase_dataloader):
        
        optim.zero_grad()
        pred = classifier(data[0].float().cuda())
        loss = CrossEntropy(pred, data[1].cuda())
        loss.backward()
        optim.step()
        
        total_train_loss.append(loss.item())
        train_acc += np.sum(np.argmax(pred.cpu().data.numpy(), axis=1) == data[1].numpy())
        train_loss += loss.item()
        
    
    #evaluation on valid set
    classifier.eval()
    with torch.no_grad():
        for i, data in enumerate(valid_dataloader):
            pred = classifier(data[0].float().cuda())
            loss = CrossEntropy(pred, data[1].cuda())
            
            total_valid_loss.append(loss.item())
            valid_acc += np.sum(np.argmax(pred.cpu().data.numpy(), axis=1) == data[1].numpy())
            valid_loss += loss.item()
            
    print("Epoch: %02d | train_acc: %6.6f train_loss: %6.6f | valid_acc: %6.6f valid_loss: %6.6f" % \
          (epoch+1, train_acc/repurchase_dataset.__len__(), train_loss/repurchase_dataset.__len__(), \
           valid_acc/valid_dataset.__len__(), valid_loss/valid_dataset.__len__()))

model_name = "repurchase.pkl"
torch.save(classifier.state_dict() , model_name)

/home/chiluen/.local/lib/python3.6/site-packages/torch/nn/modules/container.py:100: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  input = module(input)


Epoch: 01 | train_acc: 0.841967 train_loss: 0.004790 | valid_acc: 0.895137 valid_loss: 0.004195
Epoch: 02 | train_acc: 0.897466 train_loss: 0.004163 | valid_acc: 0.895661 valid_loss: 0.004170
Epoch: 03 | train_acc: 0.897545 train_loss: 0.004149 | valid_acc: 0.894822 valid_loss: 0.004171
Epoch: 04 | train_acc: 0.897650 train_loss: 0.004144 | valid_acc: 0.895504 valid_loss: 0.004162
Epoch: 05 | train_acc: 0.897754 train_loss: 0.004140 | valid_acc: 0.895346 valid_loss: 0.004163
Epoch: 06 | train_acc: 0.898580 train_loss: 0.004137 | valid_acc: 0.895084 valid_loss: 0.004164
Epoch: 07 | train_acc: 0.898724 train_loss: 0.004135 | valid_acc: 0.894665 valid_loss: 0.004161
Epoch: 08 | train_acc: 0.898868 train_loss: 0.004133 | valid_acc: 0.895346 valid_loss: 0.004165
Epoch: 09 | train_acc: 0.898881 train_loss: 0.004131 | valid_acc: 0.895189 valid_loss: 0.004164
Epoch: 10 | train_acc: 0.899143 train_loss: 0.004128 | valid_acc: 0.894613 valid_loss: 0.004167
Epoch: 11 | train_acc: 0.899523 train_lo

In [19]:
# not all model_prediction are zero
X_test_tensor = torch.tensor(X_test).float().cuda()

model_predict = classifier(X_test_tensor)
model_predict = np.argmax(model_predict.cpu().data.numpy(), axis=1)
print("True repurchase number: %d | Predict repurchase number: %d" %(sum(y_test), sum(model_predict)))

True repurchase number: 2058 | Predict repurchase number: 456
